<a href="https://colab.research.google.com/github/tsaravindh/Python_ML_NN_Projects/blob/main/final_code_Weapondetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weapon Detection Using YOLO Models

## Install Required Packages

This cell installs the required Python packages:

- `ultralytics`: provides YOLO model implementation and tools.
- `playsound`: used for playing audio alerts when weapons are detected.

In [ ]:
! pip install ultralytics
!pip install playsound


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Essential Libraries

This cell imports the necessary Python libraries for object detection and processing:

- `YOLO` – for loading and working with YOLO models using the Ultralytics library.
- `cv2` – (OpenCV) for image and video reading, processing, and display.
- `playsound` – for playing audio alerts when a weapon is detected.
- `threading` – to allow alert sounds to play in parallel with detection.
- `torch` – PyTorch framework for deep learning operations and CUDA support.
- `os` – to access environment variables and manage file paths.

In [ ]:
from ultralytics import YOLO
import cv2
from playsound import playsound
import threading
import torch
import os

## Export Python Environment

This command exports the current list of installed Python packages and their versions.

- Useful for replicating the environment using `requirements.txt`.
- Helps in sharing the environment setup with collaborators.

In [ ]:
!pip freeze > requirements.txt


## Load YOLOv5 Model

Initializes a YOLOv5 model from a pre-trained weights file.

- `YOLO('yolov5su.pt')`: loads a small YOLOv5 model for fine-tuning or inference.
- You can now perform training or prediction using this model object.

In [ ]:
model = YOLO('yolov5su.pt')

## Validate Training Data Consistency

This cell checks the training set for consistency between images and labels.

- Lists all image and label filenames (excluding extensions).
- Compares the two sets to find:
  - Images without labels
  - Labels without images
- Helps ensure all training data is usable by YOLO.

In [ ]:
image_dir = r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\images\train"
label_dir = r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\train"

image_files = set([os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')])

missing_labels = image_files - label_files
missing_images = label_files - image_files

print(f"Total training images: {len(image_files)}")
print(f"Total training labels: {len(label_files)}")
print(f"Images missing labels: {missing_labels}")
print(f"Labels missing images: {missing_images}")

## Validate Validation Data Consistency

This cell ensures that validation images and labels are properly paired.

- Detects mismatches between image files and label files.
- Prevents evaluation errors due to missing annotations.

In [ ]:
image_dir = r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\images\val"
label_dir = r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\val"

image_files = set([os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')])

missing_labels = image_files - label_files
missing_images = label_files - image_files

print(f"Total validation images: {len(image_files)}")
print(f"Total validation labels: {len(label_files)}")
print(f"Images missing labels: {missing_labels}")
print(f"Labels missing images: {missing_images}")


## Check for Label File Errors

This block validates the format and content of YOLO label files.

- Detects:
  - Empty files
  - Malformed lines
  - Invalid class IDs (greater than the total class count)
- Helps ensure data quality before training.

In [ ]:
label_dir =r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\images\train"
image_dir = r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\train"
nc = 1  # number of classes from data.yaml

bad_files = []

for file in os.listdir(label_dir):
    if not file.endswith('.txt'):
        continue
    path = os.path.join(label_dir, file)
    with open(path, 'r') as f:
        lines = f.readlines()
        if len(lines) == 0:
            bad_files.append((file, "empty"))
            continue
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                bad_files.append((file, "invalid line format"))
                break
            try:
                cls = int(parts[0])
                if cls >= nc:
                    bad_files.append((file, f"invalid class {cls}"))
                    break
            except:
                bad_files.append((file, "parse error"))
                break

print("Issues found in label files:")
for f in bad_files:
    print(f)

Issues found in label files:


## Clear CUDA Cache

This command clears unused GPU memory in PyTorch.

- Useful when switching models or after out-of-memory errors.
- Helps avoid memory fragmentation issues during training or inference.

In [ ]:
import torch
torch.cuda.empty_cache()

## Train YOLOv5 Model

This block trains the YOLOv5 model using the `.train()` method.

- Parameters include:
  - `data`: Path to the `data.yaml` file containing dataset info.
  - `epochs`: Number of training iterations.
  - `imgsz`: Input image size (typically 640).
  - `batch`: Number of images per batch.
  - `cache`: Whether to cache data for faster training.
  - `close_mosaic`: Mosaic augmentation control.
  - `half`: Use half precision (mixed precision) to save GPU memory.

In [ ]:

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# Train the model
model.train(
    data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    cache=False,
    close_mosaic=0,
    half=True
)

Ultralytics 8.3.141  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/yalla/Documents/Anaconda/capstone/weapon-detect-yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5su.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train14, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tru

train: Scanning C:\Users\yalla\Documents\Anaconda\capstone\weapon-detect-yolo\labels\train.cache... 718 images, 5 backg


val: Fast image access  (ping: 0.50.3 ms, read: 221.371.1 MB/s, size: 407.6 KB)


val: Scanning C:\Users\yalla\Documents\Anaconda\capstone\weapon-detect-yolo\labels\val.cache... 50 images, 0 background


Plotting labels to runs\detect\train14\labels.jpg... 
WARNING No module named 'matplotlib_inline'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train14
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         50         84      0.395      0.405      0.309      0.116



50 epochs completed in 0.418 hours.
Optimizer stripped from runs\detect\train14\weights\last.pt, 18.5MB
Optimizer stripped from runs\detect\train14\weights\best.pt, 18.5MB

Validating runs\detect\train14\weights\best.pt...
Ultralytics 8.3.141  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all         50         84      0.444      0.357      0.327      0.129
Speed: 0.9ms preprocess, 9.4ms inference, 0.0ms loss, 8.0ms postprocess per image
Results saved to runs\detect\train14


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001CAF85A08E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Evaluate Model Performance

Evaluates the trained YOLO model using validation data.

- The `.val()` method returns performance metrics like:
  - `precision`, `recall`, `mAP50`, `mAP50-95`
- Helpful to assess model generalization to unseen data.

In [ ]:

# Evaluate the trained model
metrics = model.val()
print(metrics)


Ultralytics 8.3.141  Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 636.6336.9 MB/s, size: 357.8 KB)


val: Scanning C:\Users\yalla\Documents\Anaconda\capstone\weapon-detect-yolo\labels\val.cache... 50 images, 0 background
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<0


                   all         50         84      0.421      0.345      0.327      0.126
Speed: 0.9ms preprocess, 35.1ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs\detect\train142
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001CB066A5BA0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,  

## Save Trained YOLOv5 Model

Saves the trained model to disk for future use.

- Format: `.pt` (PyTorch model checkpoint)
- Can be reloaded later using `YOLO('path/to/model.pt')`

In [ ]:
model.save("C:/Users/yalla/Documents/Anaconda/capstone/my_weapon_model.pt")


## Initialize YOLOv8 Model

Loads a pretrained YOLOv8 model using the small architecture (`yolov8s.pt`).

- YOLOv8 is the latest iteration in the YOLO family by Ultralytics.
- You can fine-tune this model or use it for evaluation and inference.

In [ ]:
model1 = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:00<00:00, 22.7MB/s]


## Train YOLOv8 Model

Trains the YOLOv8 model using updated training features.

- Important arguments include:
  - `data`: Path to your dataset YAML file.
  - `epochs`, `batch`, `imgsz`: typical training hyperparameters.
  - `mosaic`, `augment`: augmentation options.
  - `cache`: optionally speed up by caching images.
  - `half`: enables mixed precision (saves memory).
  - `patience`: early stopping criteria (number of epochs to wait).

In [ ]:

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# Step 2: Train the model
model1.train(
    data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,         # GPU index
    cache=False,
    close_mosaic=10,  # better generalization at the end
    half=True,        # mixed precision
    workers=4,
    patience=20,
)

Ultralytics 8.3.153  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train16, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

100%|██████████| 755k/755k [00:00<00:00, 8.35MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               


  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768,

train: Scanning C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\train... 2000 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:01<00:00, 1204.49it/s]


...

                   all        500       1077      0.337      0.179      0.162     0.0633



50 epochs completed in 0.598 hours.
Optimizer stripped from runs\detect\train16\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train16\weights\best.pt, 22.5MB

Validating runs\detect\train16\weights\best.pt...
Ultralytics 8.3.153  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:05<00:00,  6.38it/s]


                   all        500       1077       0.34      0.195      0.167      0.065
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\train16


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002820491EC50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Save Trained YOLOv8 Model

In [ ]:
model1.save(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection_yolov8s_model.pt")


In [ ]:
model2 = YOLO("yolov5su.pt")

In [ ]:
model2.train(
    data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    conf=0.001,
    close_mosaic=10,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # color aug
    flipud=0.0, fliplr=0.5,
    degrees=0.0, translate=0.1, scale=0.5,
    device=0,
    patience=20
)


In [ ]:
model.save("C:/Users/yalla/Documents/Anaconda/capstone/weapon_detection_yolov5su_newmodel.pt")

In [ ]:
from ultralytics import YOLO

# Load models
model = YOLO(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\code\runs\detect\train14\weights\best.pt")
model1 = YOLO(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\code\runs\detect\train16\weights\best.pt")
model2 = YOLO(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\code\runs\detect\train16\weights\best.pt")

# Evaluate on the same validation set
metrics = model.val(data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml", split="val")
metrics1 = model1.val(data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml", split="val")
metrics2= model2.val(data=r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\data.yaml", split="val")


# Compare key metrics
print("Model mAP50:", metrics1.box.map50)
print("Model 1 mAP50:", metrics2.box.map50)
print("Model 2 mAP50:", metrics2.box.map50)



Ultralytics 8.3.153  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs
val: Fast image access  (ping: 0.60.2 ms, read: 311.173.3 MB/s, size: 206.5 KB)


val: Scanning C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\val.cache... 500 images, 8 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  6.63it/s]


                   all        500       1077      0.549      0.399      0.385       0.18
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\val2
Ultralytics 8.3.153  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.20.0 ms, read: 544.9257.3 MB/s, size: 159.5 KB)


val: Scanning C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection\labels\val.cache... 500 images, 8 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  6.64it/s]


                   all        500       1077      0.336      0.196      0.167     0.0651
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\val3
Model 1 mAP50: 0.3846311373482039
Model 2 mAP50: 0.1668211450874083


AttributeError: 'Metric' object has no attribute 'recall'. See valid attributes below.

    Class for computing evaluation metrics for Ultralytics YOLO models.

    Attributes:
        p (list): Precision for each class. Shape: (nc,).
        r (list): Recall for each class. Shape: (nc,).
        f1 (list): F1 score for each class. Shape: (nc,).
        all_ap (list): AP scores for all classes and all IoU thresholds. Shape: (nc, 10).
        ap_class_index (list): Index of class for each AP score. Shape: (nc,).
        nc (int): Number of classes.

    Methods:
        ap50(): AP at IoU threshold of 0.5 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        ap(): AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        mp(): Mean precision of all classes. Returns: Float.
        mr(): Mean recall of all classes. Returns: Float.
        map50(): Mean AP at IoU threshold of 0.5 for all classes. Returns: Float.
      

## Load Model for Inference

Loads the final YOLOv8 model for real-time or offline prediction.

- This will be used to process input video frames and detect weapons.


In [ ]:
model_predict=YOLO(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\weapon_detection_yolov8s_model.pt")

## Real-Time Weapon Detection from Video

Uses the loaded YOLO model to process video frame-by-frame.

- Opens the webcam or video file using `cv2.VideoCapture`.
- For each frame:
  - Runs YOLO inference.
  - Draws bounding boxes on detected objects.
  - Plays an alert sound for detected weapons with confidence > 0.5.

In [ ]:


# Play alert sound in a separate thread
def play_alert():
    playsound("alert.mp3")

# Open video source
cap = cv2.VideoCapture(r"C:\Users\akram\OneDrive\Desktop\saeed\capstone\code\WhatsApp Video 2025-06-03 at 12.37.29_f6ce3043.mp4")

# Flag to avoid multiple alerts in one frame
alert_triggered = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model_predict.predict(
    frame,
    conf=0.25,        # lower for fewer FNs
    iou=0.5,          # adjust NMS
    max_det=10        # limit detections per image
)

    # Check detections for class 'weapon' (adjust name or index based on your model)
    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = results[0].names[cls_id]

        if label.lower() == "weapon" and conf > 0.5:
            # Draw bounding box and label
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            cv2.rectangle(frame, tuple(xyxy[:2]), tuple(xyxy[2:]), (0, 0, 255), 3)
            cv2.putText(frame, f"{label} ({conf:.2f})", tuple(xyxy[:2] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            if not alert_triggered:
                alert_triggered = True
                threading.Thread(target=play_alert).start()

    # Show frame
    cv2.imshow("Weapon Detection Alert", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
